In [3]:
# Configuracion para recargar módulos y librerías 
%reload_ext autoreload
%autoreload 2

In [4]:
# Configuración para plots inline
%matplotlib inline

# MAT281 - Laboratorio 9

## Aplicaciones de la Matemática en la Ingeniería

Puedes ejecutar este jupyter notebook de manera interactiva:

[![Binder](../shared/images/jupyter_binder.png)](https://mybinder.org/v2/gh/sebastiandres/mat281_m04_data_science/master?filepath=/07_lab_clustering//07_lab_clustering.ipynb)

[![Colab](../shared/images/jupyter_colab.png)](https://colab.research.google.com/github/sebastiandres/mat281_m04_data_science/blob/master///07_lab_clustering//07_lab_clustering.ipynb)

## __Intrucciones__

* Completa tus datos personales (nombre y rol USM).
* Debes enviar este .ipynb con el siguiente formato de nombre: 08_lab_clasificacion_NOMBRE_APELLIDO.ipynb con tus respuestas a alonso.ogueda@gmail.com y sebastian.flores@usm.cl .
* Se evaluará:
    - Soluciones
    - Código
    - Al presionar  `Kernel -> Restart Kernel and Run All Cells` deben ejecutarse todas las celdas sin error.
    - La escala es de 0 a 4 considerando solo valores enteros.
* __La entrega es al final de esta clase.__

__Nombre__:

__Rol__:

## Observación

***Este laboratorio utiliza la librería sklearn (oficialmente llamada [scikit learn](http://scikit-learn.org/stable/)), puesto que buscamos aplicar la técnica del clustering a datos tal como se haría en una aplicación real. El código a proveer en este laboratorio es reducido, y la nota se basará mayoritariamente en la calidad de las respuestas entregadas en los comentarios.***

## Problema: Wine Dataset

Los datos del [Wine Dataset](https://archive.ics.uci.edu/ml/datasets/Wine) son un conjunto de datos clásicos para verificar los algoritmos de clustering. 

<img src="images/wine.jpg" alt="" width="600px" align="middle"/>

Los datos corresponden a 3 cultivos diferentes de vinos de la misma región de Italia, y que han sido identificados con las etiquetas 1, 2 y 3. Para cada tipo de vino han sido realizados 13 análisis químicos:

1. Alcohol 
2. Malic acid 
3. Ash 
4. Alcalinity of ash 
5. Magnesium 
6. Total phenols 
7. Flavanoids 
8. Nonflavanoid phenols 
9. Proanthocyanins 
10. Color intensity 
11. Hue 
12. OD280/OD315 of diluted wines 
13. Proline 


La base de datos contiene 178 muestras distintas en total.

## 0. Entendimiento del dataset
Antes de leer los datos y aplicar algoritmos, resulta importante comprender la naturaleza de los datos. Los datos del wine dataset ya se encuentan en la carpeta `data/`. 

Existen 2 archivos de interés:
* `wine_data.txt` : Datos de interés.
* `wine_names.txt` : Explicación de los datos.

Lea atentamente el archivo `wine_names.txt` y responda las preguntas.

In [5]:
%%bash
cat data/wine_names.txt

1. Title of Database: Wine recognition data
	Updated Sept 21, 1998 by C.Blake : Added attribute information

2. Sources:
   (a) Forina, M. et al, PARVUS - An Extendible Package for Data
       Exploration, Classification and Correlation. Institute of Pharmaceutical
       and Food Analysis and Technologies, Via Brigata Salerno, 
       16147 Genoa, Italy.

   (b) Stefan Aeberhard, email: stefan@coral.cs.jcu.edu.au
   (c) July 1991
3. Past Usage:

   (1)
   S. Aeberhard, D. Coomans and O. de Vel,
   Comparison of Classifiers in High Dimensional Settings,
   Tech. Rep. no. 92-02, (1992), Dept. of Computer Science and Dept. of
   Mathematics and Statistics, James Cook University of North Queensland.
   (Also submitted to Technometrics).

   The data was used with many others for comparing various 
   classifiers. The classes are separable, though only RDA 
   has achieved 100% correct classification.
   (RDA : 100%, QDA 99.4%, LDA 98.9%, 1NN 96.1% (z-transformed data))
   (All results usi

## Pregunta 1.1
¿Que contiene el archivo? Describa su contenido de manera que una tercera persona, que no ha visto la descripción, pueda entenderlo.

*R:* 

## Pregunta 1.2
¿**Porqué** y **cómo** podemos usar este dataset para probar algoritmos de clustering cuando los datos han sido usados para algoritmos de clasificación?

*R:* 


## 1. Lectura de datos

Antes de proceder, miremos algunas lineas del archivo a utilizar.

In [ ]:
%%bash
head data/wine_data.txt

El siguiente código permite leer los datos desde el archivo `data/wine_data.txt` y cargarlos en un dataframe. 

In [6]:
import pandas as pd
import os

names = ["alcohol", "malic_acid", "ash", "alcalinity_of_ash", "magnesium", "total_phenols",
         "flavanoids", "nonflavanoid_phenols", "proanthocyanins", "color_intensity",
         "hue", "OD280/OD315", "proline"]
columns = ["wine_class"] + names
filename = os.path.join("data","wine_data.txt")
df = pd.read_csv(filename, names=columns, sep=",")
df.head()

,wine_class,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,OD280/OD315,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


### Pregunta 2
Complete la preparación de los datos, separando los datos en un dataframe `X` (datos a utilizar para clustering) y una serie `true_labels` (etiquetas verdaderas para cada dato de `X`).


**OBSERVACION**: La serie `true_labels` debe modificarse para que sean 0, 1 y 2 (en vez de 1, 2 y 3 como vienen en el archivo), porque el algoritmo de clustering asume que las categorías se numeran desde 0.

In [ ]:
import numpy as np

# Seleccionar X
X = df # FIX ME ¿que columnas tomar?

# Seleccionar true_labels
true_labels = df # FIX ME ¿que columna tomar?

In [ ]:
# Check X
X.head()

In [ ]:
# Check true_labels
true_labels.head()

### Pregunta 3
Utilizando la serie `true_labels` definido anteriormente, complete el código para conocer cuántas muestras son de tipo 0, de tipo 1 y de tipo 2. Compare con lo indicado en el archivo.

In [ ]:
# Es muy util saber contar valores en una serie de pandas.
# Porque permite responder esta pregunta con 1 linea
vc = 0 # FIX ME

print(vc)

## 2. Exploración de valores

Antes de realizar el clustering, deseamos revisar los datos. El siguiente código permite conocer la distribución de las mediciones para las muestras. 

In [ ]:
from matplotlib import pyplot as plt
rows, cols = 5, 3
fig1, axes1 = plt.subplots(rows, cols, figsize=(16,16))
for i in range(rows):
    for  j in range(cols):
        n = i*cols + j
        if n<13:
            ax = axes1[i][j]
            col_name = names[n]
            ax.hist(X[col_name], alpha=0.50)
            ax.set_title(col_name)
fig1.tight_layout()
plt.show()

O, aprovechando las ventajas de usar la librería pandas, 
podemos utilizar los métodos nativos para obtener el mismo resultado con mucho menor esfuerzo.

In [ ]:
from matplotlib import pyplot as plt
rows, cols = 5, 3
fig2, axes2 = plt.subplots(rows, cols, figsize=(16,16))
for n, col_name in enumerate(names):
    ax = axes2[n//cols][n%cols]
    df[col_name].hist(bins=12, alpha=0.50, ax=ax)
    ax.set_title(col_name)
plt.show()

O, incluso como no son demasiadas variables, podemos graficar todas las relaciones con scatter_matrix.

In [ ]:
# Paciencia - tomará 30 segundos.
from pandas.plotting import scatter_matrix
scatter_matrix(df, alpha=0.25, figsize=(20, 20), diagonal='hist');

## Pregunta 4

En base a la exploración de valores, usted:
1. Aplicaría el algoritmo de clustering directamente.
2. Realizaría algún tipo de normalización a los datos, y luego aplicar el algoritmo de clustering.

¿Que resulta más razonable, opción 1 u opción 2? ¿Porqué?

**Justifique su respuesta**: piense en cómo funciona K-Means.

#### Respuesta

*R:* 


## 3. Clustering Directo

A continuación se provee el código para realizar el clustering de los datos de manera directa (sin normalizar). Recuerde que el algoritmo hará predicción de clusters y no de etiquetas, por lo que la matriz de confusión no necesariamente será diagonal. Para la interpretación de la matriz de confusión, considere la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

# Parameters
n_clusters = 3

# Running the algorithm
kmeans = KMeans(n_clusters)
kmeans.fit(X)
pred_labels = kmeans.labels_

cm = confusion_matrix(true_labels, pred_labels)
print(cm)

## Pregunta 5

Ejecute el código anterior y comente los resultados. ¿Permite el clustering recobrar el agrupamiento natural de los datos? ¿Si no, porqué?

#### Respuesta

*R:* 



## 3. Normalización de los datos

Sabemos que los algoritmos suelen funcionar mejor con los datos normalizados, como se explicó en la clase de Regresión Lineal. Note que en el caso de los algoritmos de clustering, sólo es necesario normalizar la matrix `X`, ¡las etiquetas no necesitan normalizarse!

## Pregunta 6.1
Normalice los datos utilizando para obtener una nueva matriz `X_normalized_1`, cuyas columnas tengan sus datos en el rango [0,1]. 

**Observación**: Utilice sus conocimientos matemáticos y opere normalmente con `X.max()` y `X.min()`. La respuesta toma 1 línea.

In [ ]:
X_normalized_1 = X # FIX ME

In [ ]:
# Check facil con el método describe
X_normalized_1.describe().T

### Pregunta 6.2 
Reutilice el código anteriormente provisto para realizar el clustering en los datos normalizados y comente los resultados obtenidos. ¿Cuantos errores existen en total?

In [ ]:
# AGREGAR CODIGO PARA REALIZAR CLUSTERING EN X_normalized_1

Comentario a los resultados obtenidos.

*R:* 

La clasificación es perfecta para la etiquetas originales 1 y 3 (0 y 2 después de re-etiquetar). Se obtiene una clasificación  con 8 errores del total de 71 para la etiqueta 2 (1 depués de re-etiquetar). La normalización mejora muchísimo el desempeño del algoritmo de clustering.  

## Nueva normalización de datos

Como usted ya posee cierta experiencia en ajustar modelos, se pregunta si resultará mejor normalizar considerando ahora que cada columna posea media $0$ y desviación estándar $1$ para cada una de sus columnas.

### Pregunta 7.1 
Estandarice los datos para obtener una nueva matriz `X_normalized_2`, de manera que `X_normalized_2` posea media 0 y desviación estándar 1 para cada una de sus columnas. 

In [ ]:
X_mod_2 = X # FIX ME

In [ ]:
# Check facil con método describe
X_normalized_2.describe().T

### Pregunta 7.2
Reutilice el código anteriormente provisto para realizar el clustering en los datos estandarizados y comente los resultados obtenidos. ¿Cuantos errores existen en total?

In [ ]:
# AGREGAR CODIGO PARA REALIZAR CLUSTERING EN X_normalized_2

Comentario a los resultados obtenidos.

*R:*

### Pregunta 8

¿Cuál de las 3 versiones aplicadas de clustering funcionó mejor? ¿Porqué cree que sea así?

*R:* 


## Bonus Track: Regla del codo
En todos los casos hemos utilizado que el número de clusters es igual a 3. El ajuste del modelo siempre será mejor al aumentar el número de clusters, pero ello no significa que el número de clusters sea el apropiado. De hecho, si tenemos que ajustar $n$ puntos, claramente tomar $n$ clusters generaría un ajuste perfecto, pero no permitiría representar si existen realmente agrupaciones de datos.

Cuando no se conoce el número de clusters a priori, se utiliza la [regla del codo](https://jarroba.com/seleccion-del-numero-optimo-clusters/), que indica que el número más apropiado es aquel donde "cambia la pendiente" de decrecimiento de la la suma de las distancias a los clusters para cada punto, en función del número de clusters.

A continuación se provee el código para el caso de clustering sobre los datos estandarizados, leídos directamente de un archivo preparado especialmente.


In [ ]:
from sklearn.cluster import KMeans

X_mod = np.loadtxt("data/X_estandarized.txt")

clusters = range(1,20)
total_distance = []
for n_clusters in clusters:
    kmeans = KMeans(n_clusters)
    kmeans.fit(X_mod)
    pred_labels = kmeans.labels_
    centroids = kmeans.cluster_centers_
    # Get the distances
    distance_for_n = 0
    for k in range(n_clusters):
        points = X_mod[pred_labels==k]
        aux = (points - centroids[k,:])**2
        distance_for_n += (aux.sum(axis=1)**0.5).sum()
    total_distance.append(distance_for_n)

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.plot(clusters, total_distance, 'rs')
plt.xlim(min(clusters)-1, max(clusters)+1)
plt.show()